### Pakage importation

In [1]:
import pandas as pd
import nltk
import numpy as np
from bs4 import BeautifulSoup
import stop_words
import re, sys
import spacy
import pickle
from gensim.models import Phrases, phrases
from gensim import models, corpora
import pyLDAvis.gensim
from gensim.models import coherencemodel
from gensim.models.coherencemodel import CoherenceModel
from time import sleep
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm, tqdm_notebook # progress bars in Jupyter
tqdm.pandas()
tqdm_notebook()

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

0it [00:00, ?it/s]

### Importing Data Set 

In [2]:
data = pd.read_csv('QueryResults.csv')
data.dropna(subset=['Tags'], inplace=True)

In [3]:
data.shape

(5522, 22)

### Preprocessing

#### Dealing with Tag

In [4]:
# Def function for regular expressions
def reg_function(x):
    tokenizer = nltk.RegexpTokenizer(r'<.*?>')
    if x == x:
        x = tokenizer.tokenize(x.lower())
        for tag in range(len(x)):
            x[tag] = x[tag].strip('>').strip('<')
    return x

def normalize(data, col):
    #data[col].dropna(inplace=True)
    data['token_tag'] = data[col].apply(reg_function)
#     data[['token_tag']].dropna(inplace=True)
    return data[['token_tag']].head()

In [5]:
normalize(data, 'Tags')

,token_tag
0,"[c#, floating-point, type-conversion, double, decimal]"
1,"[html, css, css3, internet-explorer-7]"
3,"[c#, .net, datetime]"
4,"[c#, datetime, time, datediff, relative-time-span]"
6,"[javascript, html, browser, timezone, timezoneoffset]"


In [6]:
def load_obj(name):
    print(name, 'loaded')
    with open(name, 'rb') as f:
        return pickle.load(f)
#-------------------------------------------------------------------------------------------------------
max_features_tags = 1500

# Liste de tous les tags
top_tags = load_obj('top_tags')

# Liste des tags
max_tags = []
for i in range(0, max_features_tags):
    max_tags.append(top_tags[i][0])

# Méthode pour enlever les tags hors de la liste officielle de classification
def cleanTags(p_val):
    ret = []
    for word in str(p_val).split():
        if word in max_tags:
            ret.append(word)
    return " ".join(ret)

def trans(val):
    val = " ".join(str(x) for x in val)
    return val
#------------------------------------------------------------------------------------------------------

def delete_nonrelevant_tag(data):
    # On enlève tous les tags qui ne font pas parti du top choisi
    data['tokens_tag_true'] = data['token_tag'].apply(trans).apply(cleanTags)

    # On enlève de l'entrainement les données qui se retrouvent sans target
    data = data[data['tokens_tag_true'].str.len()>0]
    print(data.shape)

top_tags loaded


In [8]:
delete_nonrelevant_tag(data)

(5479, 24)


In [10]:
data[['token_tag']]

,token_tag
0,"[c#, floating-point, type-conversion, double, decimal]"
1,"[html, css, css3, internet-explorer-7]"
3,"[c#, .net, datetime]"
4,"[c#, datetime, time, datediff, relative-time-span]"
6,"[javascript, html, browser, timezone, timezoneoffset]"
7,"[.net, math]"
8,"[c#, linq, web-services, .net-3.5]"
9,"[mysql, database, binary-data, data-storage]"
11,"[performance, algorithm, language-agnostic, unix, pi]"
14,"[mysql, database, triggers]"


#### Dealing with Body

In [11]:
# Merging Body and Title
def merge(data, col1, col2):
    data['Question'] = data[col1]+' '+data[col2]
    return data[['Question']].head()

In [12]:
merge(data, 'Body', 'Title')

,Question
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n Convert Decimal to Double?"
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n Percentage width child element in absolutely positioned parent on Internet Explorer 7"
3,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>\n How do I calculate someone's age in C#?"
4,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n\n<ul>\n<li>2 hours ago</li>\n<li>3 days ago</li>\n<li>a month ago</li>\n</ul>\n Calculate relative time in C#"
6,<p>Is there any standard way for a Web Server to be able to determine a user's timezone within a web page? </p>\n\n<p>Perhaps from an HTTP header or part of the user-agent string?</p>\n Determine a User's Timezone


In [13]:
# Cleaning text
def virer(a,str1,str2): # Delete code windows
    a=a.split(str1)
    l=[]
    for i in range(len(a)):
        b=a[i].split(str2)
        l.append(b)
    for i in range(1,len(l)):
        l[i].remove(l[i][0])  
    s=""
    for i in range(len(l)):
        s+=l[i][0]
    return s

def get_text(x):
    x = str(x)
    x=virer(x, "<code>","</code>")
#     x = virer(x,"<blockquote>","</blockquote>")
    x = BeautifulSoup(x, 'lxml').get_text() # Get text in htlm with beautifulsoup
    tokenizer = nltk.RegexpTokenizer(r'\w+') # Use regular expression to delete \n
    x = tokenizer.tokenize(x.lower()) # apply on x by tranforming x in lower character
    return x

# Creating token clean
def tokenization(data, col):
    data['token_questions'] = data[col].apply(get_text)
    sw = stop_words.get_stop_words(language='en')
    data['tokens_clean'] = data.token_questions.map(
    lambda tok: [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(tok)))
                 if t.lower() not in sw and len(t) > 1]
    )
    return data[['tokens_clean']].head()

# Lemarization
def lematization(data, col):
    lem = spacy.load('en_core_web_md')
    data['tokens_clean_lemma'] = data[col].progress_map(
    lambda x: [tok.lemma_ for tok in lem(' '.join(x))])
    return data[['tokens_clean_lemma']].head()

In [14]:
tokenization(data, 'Question')

,tokens_clean
0,"[want, use, track, bar, change, form, opacity, code, build, application, gives, following, error, implicitly, convert, type, tried, using, control, doesn, work, code, worked, fine, past, vb, net, project, convert, decimal, double]"
1,"[absolutely, positioned, containing, several, children, one, relatively, positioned, use, percentage, based, width, child, collapses, width, internet, explorer, firefox, safari, use, pixel, width, works, parent, relatively, positioned, percentage, width, child, works, something, missing, easy, fix, besides, pixel, based, width, child, area, css, specification, covers, percentage, width, child, element, absolutely, positioned, parent, internet, explorer]"
3,"[given, representing, person, birthday, calculate, age, years, calculate, someone, age]"
4,"[given, specific, value, display, relative, time, like, hours, ago, days, ago, month, ago, calculate, relative, time]"
6,"[standard, way, web, server, able, determine, user, timezone, within, web, page, perhaps, http, header, part, user, agent, string, determine, user, timezone]"


In [15]:
lematization(data, 'tokens_clean')

100%|██████████████████████████████████████████████████████████████████████████████| 5522/5522 [02:18<00:00, 39.94it/s]


,tokens_clean_lemma
0,"[want, use, track, bar, change, form, opacity, code, build, application, give, follow, error, implicitly, convert, type, try, use, control, doesn, work, code, work, fine, past, vb, net, project, convert, decimal, double]"
1,"[absolutely, position, contain, several, child, one, relatively, position, use, percentage, base, width, child, collapse, width, internet, explorer, firefox, safari, use, pixel, width, work, parent, relatively, position, percentage, width, child, work, something, miss, easy, fix, besides, pixel, base, width, child, area, css, specification, cover, percentage, width, child, element, absolutely, position, parent, internet, explorer]"
3,"[give, represent, person, birthday, calculate, age, year, calculate, someone, age]"
4,"[give, specific, value, display, relative, time, like, hour, ago, day, ago, month, ago, calculate, relative, time]"
6,"[standard, way, web, server, able, determine, user, timezone, within, web, page, perhaps, http, header, part, user, agent, string, determine, user, timezone]"


In [16]:
def pass_list(df, col):

    for x in col:
        l=[]
        for j in range(len(df)):
            l.append(str(df[x].loc[j]).replace('[',"").replace(']',"").replace("'","").split(', '))
        df[x]=l

In [17]:
# pass_list(data, ['token_tag','tokens_clean','tokens_clean_lemma'])

### TF-IDF bag of words

In [18]:
# Train, Test = train_test_split(data, test_size=0.3)

# X_train = Train['tokens_clean_lemma']
# X_test = Test['tokens_clean_lemma']
# y_train = Train['token_tag'].apply(trans)
# y_test = Test['token_tag'].apply(trans)

Train, Test = train_test_split(data, test_size=0.3) # Spliting data to avoid data leakage
# Train['token_tag'] = Train['token_tag'].apply(trans) # transform token tag
Train['tokens_clean_lemma'] = Train['tokens_clean_lemma'].apply(trans) # Transform token lemma
corpus = Train['tokens_clean_lemma'].values
vectorizer = TfidfVectorizer(ngram_range=(1, 1)) # Creating tf-idf matrix using unigram and bigram
matrix = vectorizer.fit_transform(corpus) 
matrix_ = matrix.todense()
tf_idfmatrix = pd.DataFrame(data=matrix_,columns=vectorizer.get_feature_names(), index=Train.index)
tf_idfmatrix = pd.concat([tf_idfmatrix, Train['tokens_tag_true']], ignore_index=False, axis=1)

tf_idfmatrix.head()

,aa,aad,aardvark,ab,abandon,abap,abbreviate,abbreviation,abbriviate,abc,...,éàè,öäü,сaught,сonvenient,אין,בעברית,תמיכה,罗马市,로마,tokens_tag_true
24412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,asp.net asp-classic
9676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sql oracle plsql
24480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c# asp.net sms
18448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c# winforms
7241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,php ajax authentication


In [19]:
a = vectorizer.transform(Test['tokens_clean_lemma'].apply(trans).values)
a.todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [20]:
a.shape

(1657, 11130)

### Definied metrics: Pecision @K

In [23]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

### Predictive model

#### Splitting and binarization

In [24]:
# Splitting data in training and test set
X_train, X_test, y_train, y_test = train_test_split(
    tf_idfmatrix.iloc[:, :-1].values, tf_idfmatrix['tokens_tag_true'], test_size=0.3)

#     Target binarizing
mlb = preprocessing.MultiLabelBinarizer(classes=max_tags)
mlb.fit([w.split() for w in y_train])
y_train_mlb = mlb.transform([w.split() for w in y_train])
y_test_mlb = mlb.transform([w.split() for w in y_test])

In [25]:
y_train_mlb[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [26]:
# Méthode pour récupérer les meilleurs tags en fonction du pourcentage de la prédiction
def get_best_tags(clf, X, lb, n_tags=5, b_save=False, b_Dir=''):
    decfun = []
    if hasattr(clf, 'decision_function'):
        decfun = clf.decision_function(X)
    elif hasattr(clf, 'predict_proba'):
        decfun = clf.predict_proba(X)
    else:
        return None

    best_tags = np.argsort(decfun)[:, :-(n_tags+1): -1]

    if b_save:
        # Sauvegarde du modèle pour pouvoir le rejouer dans l'API
        save_obj(clf, b_Dir + 'Classifier')
        save_obj(lb, b_Dir + 'MultiLabelBinarizer')
        
    return lb.classes_[best_tags]

#### Mutinomial NB

In [27]:
m_NB = OneVsRestClassifier(MultinomialNB(alpha=0.001))
m_NB.fit(X_train, y_train_mlb)
y_pred_NB = mlb.classes_[np.argsort(m_NB.predict_proba(X_test))][:, :-(200+1): -1]
c = mlb.inverse_transform(y_test_mlb)
y_ij = [list(elem) for elem in c]
print(mapk(y_ij, y_pred_NB, k=5))

0.271211925287


#### Linear SVD

In [29]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV

# SVM = svm.LinearSVC(multi_class='crammer_singer')
m_SVM = OneVsRestClassifier(svm.LinearSVC(multi_class='crammer_singer')).fit(X_train, y_train_mlb)
y_pred_SVM = m_SVM.predict(X_test)
y_pred=mlb.inverse_transform(y_pred_SVM)
y_pred_1= [list(elem) for elem in y_pred]
print(mapk(y_ij, y_pred_1, k=3))

0.228807471264


### Word Embedding

#### Creating corpus

In [30]:
corpus_w = data['tokens_clean_lemma'].values.tolist()

#### Training word embedding with word2vec

In [31]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(corpus_w, min_count=1, size=10)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
model.most_similar('convert')

[('inner', 0.9898452758789062),
 ('concatenate', 0.9884477853775024),
 ('serialize', 0.9862602949142456),
 ('recursive', 0.9843656420707703),
 ('mapping', 0.9843652248382568),
 ('fortunately', 0.9832175970077515),
 ('datatable', 0.980750322341919),
 ('midnight', 0.9802878499031067),
 ('entity', 0.9800678491592407),
 ('dataset', 0.9800617694854736)]

#### Using word2vec as features for supervized model

In [32]:
#Class for simple word2vec 

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.items())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
# Class for word2vec+tf_idf

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.items())))

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [33]:
# Transform corpus into embedding matrix
Embedding_matrix = MeanEmbeddingVectorizer(w2v).transform(corpus_w)
Embedding_matrix.shape

(5522, 10)

In [34]:
# Splitting data in training and test set
from sklearn.model_selection import train_test_split
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(Embedding_matrix, data['tokens_tag_true'], test_size=0.3)

# Target binarizing
from sklearn import preprocessing
mlb = preprocessing.MultiLabelBinarizer(classes=max_tags)
mlb.fit([w.split() for w in y_train_2])

y_train_mlb_2 = mlb.transform([w.split() for w in y_train_2])
y_test_mlb_2 = mlb.transform([w.split() for w in y_test_2])

In [35]:
#### MNB and SVM do not support negative values
from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import NMF
# nmf = NMF(n_components=10)
scale_up = MinMaxScaler()
X_train_2 = scale_up.fit_transform(X_train_2)
X_test_2 = scale_up.transform(X_test_2)

#### MNB

In [36]:
m_NB_2 = OneVsRestClassifier(MultinomialNB(alpha=0.001))
m_NB_2.fit(X_train_2, y_train_mlb_2)
y_pred_NB_2 = mlb.classes_[np.argsort(m_NB_2.predict_proba(X_test_2))][:, :-(200+1): -1]
c_2 = mlb.inverse_transform(y_test_mlb_2)
y_tes = [list(elem) for elem in c]
mapk(y_tes, y_pred_NB_2, k=5)

0.089751197318007653

#### Linear SVC

In [37]:
m_SVM_2 = OneVsRestClassifier(svm.LinearSVC(multi_class='crammer_singer')).fit(X_train_2, y_train_mlb_2)
y_pred_SVM = mlb.classes_[np.argsort(m_SVM_2.decision_function(X_test_2))]#[:, :-(200+1): -1]
mapk(y_tes, y_pred_SVM, k=5)
# print(apk(y_ij, y_pred_SVM, k=5))

0.018739224137931033

### Saving the Model using Pickle

#### Best model

In [39]:
# Fit on data with best model
Best_model = OneVsRestClassifier(MultinomialNB(alpha=0.001))
Best_model.fit(X_train, y_train_mlb)
y_pred_NB = m_NB.predict_proba(X_test)

In [40]:
Best_model.predict_proba(X_test)

array([[  4.32512373e-02,   6.88883790e-03,   9.32046752e-04, ...,
          1.89477607e-06,   1.89477607e-06,   2.55104261e-06],
       [  2.19849260e-02,   6.11721210e-02,   5.00858467e-03, ...,
          9.14875746e-06,   9.14875746e-06,   1.21452426e-05],
       [  1.47286081e-02,   3.88068910e-03,   1.23372567e-03, ...,
          9.82496562e-09,   9.82496562e-09,   1.24767066e-08],
       ..., 
       [  4.17147023e-02,   2.14875410e-02,   2.56863228e-03, ...,
          2.14211225e-06,   2.14211225e-06,   6.40548392e-07],
       [  7.76865744e-02,   1.71797483e-01,   5.37091989e-02, ...,
          1.45345108e-04,   1.45345108e-04,   1.54495574e-04],
       [  3.49304480e-03,   1.00338207e-02,   2.98756182e-02, ...,
          3.20149160e-05,   3.20149160e-05,   4.98726608e-05]])

In [41]:
best_tags = np.argsort(Best_model.predict_proba(X_test))[:, :-(5+1): -1]

In [42]:
best_tags

array([[  0,   3,  32,  37,  63],
       [  3,  26,   1,  21,  10],
       [  4,  76,  11,   0,  18],
       ..., 
       [  0,   7,   3,   1,  49],
       [  1,  12,   3, 256,   0],
       [ 27,   3, 192,  57,  29]], dtype=int64)

In [43]:
mlb.classes_[best_tags]

array([['c#', 'java', 'security', 'sql-server-2005', 'linq-to-sql'],
       ['java', 'vb.net', '.net', 'winforms', 'visual-studio'],
       ['sql-server', 'memory', 'database', 'c#', 'mysql'],
       ..., 
       ['c#', 'php', 'java', '.net', 'string'],
       ['.net', 'windows', 'java', 'hosting', 'c#'],
       ['macos', 'java', 'process', 'iis', 'user-interface']], dtype=object)

#### Pickle model

In [44]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(vectorizer, m_NB)
pipe

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...sifier(estimator=MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True),
          n_jobs=1))])

In [45]:
import pickle
with open('Best_model.pkl', 'wb') as fid:
    pickle.dump(pipe, fid,2) 

In [46]:
import pickle
with open('Best_model_1.pkl', 'wb') as fid:
    pickle.dump(pipe, fid,2) 

In [47]:
with open('mlb.pkl', 'wb') as fid:
    pickle.dump(mlb, fid,2) 

In [48]:
max_tags_dict = dict(zip(max_tags,range(len(max_tags))))

with open('tags_api', 'wb') as fid:
    pickle.dump(max_tags_dict, fid,2)  

#### Test of model

In [49]:
Title = 'Plot beautiful graph with seaborn'
Body = 'Hello, I want to know how to plot beautiful graph on jupyter notebook using seaborn. And another questions is focused on difference between matplolib and seaborn. What would be the best ? Thanks'

In [50]:
tl="Wordpress Elementor: display Product Variation as checkboxes"
cd="Our Product page is currently using the Elementor Product > Add to cart widget, which adds a Select field for the Product Variation ‘sizes’, I would like to display size variations as checkboxes, how can I do this?"

In [51]:
Body

'Hello, I want to know how to plot beautiful graph on jupyter notebook using seaborn. And another questions is focused on difference between matplolib and seaborn. What would be the best ? Thanks'

In [52]:
Title

'Plot beautiful graph with seaborn'

In [53]:
data_1=pd.DataFrame({'Title':[tl], 'Body':[cd]})
data_1

,Body,Title
0,"Our Product page is currently using the Elementor Product > Add to cart widget, which adds a Select field for the Product Variation ‘sizes’, I would like to display size variations as checkboxes, how can I do this?",Wordpress Elementor: display Product Variation as checkboxes


In [54]:
merge(data_1,'Body', 'Title')

,Question
0,"Our Product page is currently using the Elementor Product > Add to cart widget, which adds a Select field for the Product Variation ‘sizes’, I would like to display size variations as checkboxes, how can I do this? Wordpress Elementor: display Product Variation as checkboxes"


In [55]:
tokenization(data_1, 'Question')

,tokens_clean
0,"[product, page, currently, using, elementor, product, add, cart, widget, adds, select, field, product, variation, sizes, like, display, size, variations, checkboxes, can, wordpress, elementor, display, product, variation, checkboxes]"


In [56]:
lematization(data_1, 'tokens_clean')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.45it/s]


,tokens_clean_lemma
0,"[product, page, currently, use, elementor, product, add, cart, widget, add, select, field, product, variation, size, like, display, size, variation, checkbox, can, wordpress, elementor, display, product, variation, checkbox]"


In [57]:
data_1['tokens_clean_lemma']

0    [product, page, currently, use, elementor, product, add, cart, widget, add, select, field, product, variation, size, like, display, size, variation, checkbox, can, wordpress, elementor, display, product, variation, checkbox]
Name: tokens_clean_lemma, dtype: object

In [58]:
Tags_predict=pipe.predict_proba(data_1['tokens_clean_lemma'].apply(trans))

In [59]:
best_tags_1 = np.argsort(Tags_predict)[:, :-(5+1): -1]

In [60]:
mlb.classes_[best_tags_1]

array([['user-interface', 'html', 'javascript', 'winforms', 'jquery']], dtype=object)